In [19]:
# Cell 0: IMPORTS
import cv2
import numpy as np

In [20]:
# Carregar imagem de teste
imagem = cv2.imread("../../dataset/Apple/3_100.jpg")
cinza = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)

In [ ]:
# Cell 2: HISTOGRAM OF ORIENTED GRADIENTS (HOG)
def calcular_gradientes(imagem):
    """
    Calcula os gradientes da imagem usando filtros Sobel.
    
    Args:
        imagem: Imagem em escala de cinza
        
    Returns:
        tuple: (magnitude, angulo) dos gradientes
    """
    grad_x = cv2.Sobel(imagem, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(imagem, cv2.CV_64F, 0, 1, ksize=3)
    
    magnitude = np.sqrt(grad_x**2 + grad_y**2)
    angulo = np.arctan2(grad_y, grad_x)
    angulo = np.degrees(angulo) % 180
    
    return magnitude, angulo

def calcular_hog_array(imagem, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
    """
    Calcula o Histogram of Oriented Gradients (HOG) de uma imagem.
    
    Args:
        imagem: Imagem em escala de cinza
        orientations: Número de orientações do gradiente (padrão: 9)
        pixels_per_cell: Tamanho da célula em pixels (padrão: (8, 8))
        cells_per_block: Número de células por bloco (padrão: (2, 2))
        
    Returns:
        list: Vetor HOG normalizado
    """
    altura, largura = imagem.shape
    cell_h, cell_w = pixels_per_cell
    block_h, block_w = cells_per_block

    magnitude, angulo = calcular_gradientes(imagem)
    n_cells_h = altura // cell_h
    n_cells_w = largura // cell_w
    cell_histograms = np.zeros((n_cells_h, n_cells_w, orientations))
    bin_size = 180 / orientations

    for i in range(n_cells_h):
        for j in range(n_cells_w):
            y_start, y_end = i * cell_h, (i + 1) * cell_h
            x_start, x_end = j * cell_w, (j + 1) * cell_w
            cell_mag = magnitude[y_start:y_end, x_start:x_end]
            cell_ang = angulo[y_start:y_end, x_start:x_end]
            for y in range(cell_h):
                for x in range(cell_w):
                    mag_val = cell_mag[y, x]
                    ang_val = cell_ang[y, x]
                    bin_idx = ang_val / bin_size
                    bin_low = int(bin_idx) % orientations
                    bin_high = (bin_low + 1) % orientations
                    weight = bin_idx - int(bin_idx)
                    cell_histograms[i, j, bin_low] += mag_val * (1 - weight)
                    cell_histograms[i, j, bin_high] += mag_val * weight

    n_blocks_h = n_cells_h - block_h + 1
    n_blocks_w = n_cells_w - block_w + 1
    hog_features = []

    for i in range(n_blocks_h):
        for j in range(n_blocks_w):
            block = cell_histograms[i:i+block_h, j:j+block_w, :]
            block_vector = block.flatten()
            norm = np.sqrt(np.sum(block_vector**2) + 1e-6)
            block_vector = block_vector / norm
            hog_features.extend(block_vector)

    hog_features = np.array(hog_features, dtype=np.float32)
    norm_global = np.linalg.norm(hog_features)
    hog_features_normalized = hog_features / (norm_global + 1e-8)

    return hog_features_normalized.tolist()

# def calcular_hog_completo(imagem, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
#     """
#     Calcula HOG com estatísticas e metadados completos para ML.
    
#     Args:
#         imagem: Imagem em escala de cinza
#         orientations: Número de orientações do gradiente (padrão: 9)
#         pixels_per_cell: Tamanho da célula em pixels (padrão: (8, 8))
#         cells_per_block: Número de células por bloco (padrão: (2, 2))
        
#     Returns:
#         dict: Dados completos para ML (features, stats, params, shape_info)
#     """
#     altura, largura = imagem.shape
    
#     hog_features_array = calcular_hog_array(imagem, orientations, pixels_per_cell, cells_per_block)
#     hog_features = np.array(hog_features_array, dtype=np.float32)
    
#     # Calcular estatísticas
#     stats = {
#         'mean': float(hog_features.mean()),
#         'std': float(hog_features.std()),
#         'sum': float(hog_features.sum()),
#         'min': float(hog_features.min()),
#         'max': float(hog_features.max()),
#         'l2_norm': float(np.linalg.norm(hog_features))
#     }
    
#     # Parâmetros utilizados
#     params = {
#         'orientations': orientations,
#         'pixels_per_cell': pixels_per_cell,
#         'cells_per_block': cells_per_block,
#         'bin_size': 180 / orientations
#     }
    
#     # Informações sobre dimensões
#     cell_h, cell_w = pixels_per_cell
#     block_h, block_w = cells_per_block
#     n_cells_h = altura // cell_h
#     n_cells_w = largura // cell_w
#     n_blocks_h = n_cells_h - block_h + 1
#     n_blocks_w = n_cells_w - block_w + 1
    
#     shape_info = {
#         'image_shape': (altura, largura),
#         'n_cells': (n_cells_h, n_cells_w),
#         'n_blocks': (n_blocks_h, n_blocks_w),
#         'features_length': len(hog_features),
#         'expected_features': n_blocks_h * n_blocks_w * block_h * block_w * orientations
#     }
    
#     return {
#         'features': hog_features_array,  # Lista para JSON
#         'stats': stats,
#         'params': params,
#         'shape_info': shape_info
#     }

# Exemplo de uso:
hog_array = calcular_hog_array(cinza)
# print(f"Array HOG gerado com {len(hog_array)} valores.")

# resultado_completo = calcular_hog_completo(cinza)
# print(f"HOG completo calculado:")
# print(f"  - Features: {len(resultado_completo['features'])} valores")
# print(f"  - Estatísticas: {resultado_completo['stats']}")
# print(f"  - Parâmetros: {resultado_completo['params']}")

Array HOG gerado com 4356 valores.
HOG completo calculado:
  - Features: 4356 valores
  - Estatísticas: {'mean': 0.009501689113676548, 'std': 0.011801961809396744, 'sum': 41.38935852050781, 'min': 0.0, 'max': 0.08940967917442322, 'l2_norm': 1.0}
  - Parâmetros: {'orientations': 9, 'pixels_per_cell': (8, 8), 'cells_per_block': (2, 2), 'bin_size': 20.0}
